In [1]:
import pandas as pd
import mibian
import numpy as np
import datetime
import statsmodels.api as sm
from statsmodels.formula.api import ols

/tmp/ipykernel_328073/1693152928.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
eth = pd.read_csv('../data processing/deribit lyra comparison/eth.csv')
btc = pd.read_csv('../data processing/deribit lyra comparison/btc.csv')

In [ ]:
def add_greeks_and_charachtaristics_for_regression(data):
    data['expiry'] = data['expiry'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d"))
    data['expiry'] = pd.to_datetime(data['expiry'])

    data['date'] = pd.to_datetime(data['date'])

    data['Maturity'] = data['time_to_maturity'] * 365
    data['Mness'] = np.where(data['type'] == 'call', data['selected_spot_price'] / data['strike'],
                            data['strike'] / data['selected_spot_price'])

    data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
    data['option_id'] = data['instrument'].astype('category').cat.codes
    data['time_id'] = pd.to_datetime(data['date']).dt.date.astype('category').cat.codes
    data['Diff_IV'] = data['lyra_IV'] - data['selected_deribit_IV']
    # Add binary variable for Call option
    data['Call'] = (data['type'] == 'call').astype(int)


    # Calculate Rho, Vega, Gamma and Theta for each option entry
    for index, row in data.iterrows():
        # Assuming 365 days per year and interest rates are expressed as percentages
        bs = mibian.BS([row['selected_spot_price'], row['strike'], row['interest_rate'], row['Maturity']], 
                       volatility=row['lyra_IV'])
        data.at[index, 'Rho'] = bs.callRho if row['type'] == 'call' else bs.putRho
        data.at[index, 'Gamma'] = bs.gamma
        data.at[index, 'Theta'] = bs.callTheta if row['type'] == 'call' else bs.putTheta
        data.at[index, 'exerciceProbability'] = bs.exerciceProbability
        data.at[index, 'delta'] = abs(bs.callDelta) if row['type'] == 'call' else abs(bs.putDelta)
        data.at[index, 'vega'] = bs.vega

    return data

eth = add_greeks_and_charachtaristics_for_regression(eth)
btc = add_greeks_and_charachtaristics_for_regression(btc)

eth.to_csv('eth-regression.csv', index=False)
btc.to_csv('btc-regression.csv', index=False)

This section creates the regression panel table

In [5]:
eth = pd.read_csv('eth-regression.csv')
btc = pd.read_csv('btc-regression.csv')

def extract_model_info(model):
    """Extracts coefficients, p-values, R-squared, and adjusted R-squared from a regression model."""
    return model.params.to_dict(), model.pvalues.to_dict(), model.rsquared, model.rsquared_adj

def get_filtered(data, quantile):
    """Filters data based on quantile thresholds for 'option_id' and 'time_id'."""
    # any filteration can be done here, now we ignore it
    # filtered_data = data[data['Mness'] < 2]
    return data

def safe_format(value, fmt):
    try:
        return fmt.format(float(value))
    except (ValueError, TypeError):
        return 'n/a'

def generate_latex_table(data, filtered_data, prefix):
    """Generates a LaTeX table from regression model summaries."""
    # Model 1: Basic model
    model = ols('Diff_IV ~ Call + Maturity + Mness', data=filtered_data).fit()
    model_time_fe = ols('Diff_IV ~ Call + Maturity + Mness + C(time_id)', data=filtered_data).fit()
    model_option_fe = ols('Diff_IV ~ Call + Maturity + Mness + C(option_id)', data=filtered_data).fit()
    
    # Model 2: Including Greeks
    model_2 = ols('Diff_IV ~ delta + vega', data=filtered_data).fit()
    model_2_time_fe = ols('Diff_IV ~ delta + vega + C(time_id)', data=filtered_data).fit()
    model_2_option_fe = ols('Diff_IV ~ delta + vega + C(option_id)', data=filtered_data).fit()

    # Extract model information
    coef_1, pvals_1, r2_1, adj_r2_1 = extract_model_info(model)
    coef_1_time, pvals_1_time, r2_1_time, adj_r2_1_time = extract_model_info(model_time_fe)
    coef_1_option, pvals_1_option, r2_1_option, adj_r2_1_option = extract_model_info(model_option_fe)
    
    coef_2, pvals_2, r2_2, adj_r2_2 = extract_model_info(model_2)
    coef_2_time, pvals_2_time, r2_2_time, adj_r2_2_time = extract_model_info(model_2_time_fe)
    coef_2_option, pvals_2_option, r2_2_option, adj_r2_2_option = extract_model_info(model_2_option_fe)

    # Creating the LaTeX table
    latex_table = f"""
\\begin{{table}}[ht]
\\centering
\\begin{{tabular}}{{@{{}}lcccccc@{{}}}}
\\Variable & \\multicolumn{{3}}{{c}}{{Diff IV (Options Characteristics)}} & \\multicolumn{{3}}{{c}}{{Diff IV (Option Greeks)}} \\\\
\\cmidrule(r){{2-4}} \\cmidrule(l){{5-7}}
& & & & & & \\\\
\\midrule
Intercept & {safe_format(coef_1.get('Intercept', 'n/a'), '{:.4f}')} & {safe_format(coef_1_time.get('Intercept', 'n/a'), '{:.4f}')} & {safe_format(coef_1_option.get('Intercept', 'n/a'), '{:.4f}')} & {safe_format(coef_2.get('Intercept', 'n/a'), '{:.4f}')} & {safe_format(coef_2_time.get('Intercept', 'n/a'), '{:.4f}')} & {safe_format(coef_2_option.get('Intercept', 'n/a'), '{:.4f}')} \\\\
& ({safe_format(pvals_1.get('Intercept', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_time.get('Intercept', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_option.get('Intercept', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_2.get('Intercept', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_2_time.get('Intercept', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_2_option.get('Intercept', 'n/a'), '{:.3f}')}) \\\\
\\midrule
Call & {safe_format(coef_1.get('Call', 'n/a'), '{:.4f}')} & {safe_format(coef_1_time.get('Call', 'n/a'), '{:.4f}')} & {safe_format(coef_1_option.get('Call', 'n/a'), '{:.4f}')} &  &  &  \\\\
& ({safe_format(pvals_1.get('Call', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_time.get('Call', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_option.get('Call', 'n/a'), '{:.3f}')}) &  &  &  \\\\
Maturity & {safe_format(coef_1.get('Maturity', 'n/a'), '{:.4f}')} & {safe_format(coef_1_time.get('Maturity', 'n/a'), '{:.4f}')} & {safe_format(coef_1_option.get('Maturity', 'n/a'), '{:.4f}')} &  &  &  \\\\
& ({safe_format(pvals_1.get('Maturity', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_time.get('Maturity', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_option.get('Maturity', 'n/a'), '{:.3f}')}) &  &  &  \\\\
Mness & {safe_format(coef_1.get('Mness', 'n/a'), '{:.4f}')} & {safe_format(coef_1_time.get('Mness', 'n/a'), '{:.4f}')} & {safe_format(coef_1_option.get('Mness', 'n/a'), '{:.4f}')} &  &  &  \\\\
& ({safe_format(pvals_1.get('Mness', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_time.get('Mness', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_1_option.get('Mness', 'n/a'), '{:.3f}')}) &  &  &  \\\\
\\midrule
Delta &  &  &  & {safe_format(coef_2.get('delta', 'n/a'), '{:.4f}')} & {safe_format(coef_2_time.get('delta', 'n/a'), '{:.4f}')} & {safe_format(coef_2_option.get('delta', 'n/a'), '{:.4f}')} \\\\
&  &  &  & ({safe_format(pvals_2.get('delta', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_2_time.get('delta', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_2_option.get('delta', 'n/a'), '{:.3f}')}) \\\\
Vega &  &  &  & {safe_format(coef_2.get('vega', 'n/a'), '{:.4f}')} & {safe_format(coef_2_time.get('vega', 'n/a'), '{:.4f}')} & {safe_format(coef_2_option.get('vega', 'n/a'), '{:.4f}')} \\\\
&  &  &  & ({safe_format(pvals_2.get('vega', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_2_time.get('vega', 'n/a'), '{:.3f}')}) & ({safe_format(pvals_2_option.get('vega', 'n/a'), '{:.3f}')}) \\\\
\\midrule
R-squared & {safe_format(r2_1, '{:.3f}')} & {safe_format(r2_1_time, '{:.3f}')} & {safe_format(r2_1_option, '{:.3f}')} & {safe_format(r2_2, '{:.3f}')} & {safe_format(r2_2_time, '{:.3f}')} & {safe_format(r2_2_option, '{:.3f}')} \\\\
Adj. R-squared & {safe_format(adj_r2_1, '{:.3f}')} & {safe_format(adj_r2_1_time, '{:.3f}')} & {safe_format(adj_r2_1_option, '{:.3f}')} & {safe_format(adj_r2_2, '{:.3f}')} & {safe_format(adj_r2_2_time, '{:.3f}')} & {safe_format(adj_r2_2_option, '{:.3f}')} \\\\
Observations & {len(filtered_data)} & {len(filtered_data)} & {len(filtered_data)} & {len(filtered_data)} & {len(filtered_data)} & {len(filtered_data)} \\\\
Entities & {filtered_data['option_id'].nunique()} & & {filtered_data['option_id'].nunique()} & & & {filtered_data['option_id'].nunique()} \\\\
Time Periods & {filtered_data['time_id'].nunique()} & & {filtered_data['time_id'].nunique()} & & & {filtered_data['time_id'].nunique()} \\\\
Option-Id FE & No & Yes & Yes & No & No & Yes \\\\
Time FE & No & Yes & No & No & Yes & No \\\\
\\bottomrule
\\end{{tabular}}
\\caption{{Regression Results – IV – On-Chain vs. Off-Chain – {prefix.upper()} – Trades}}
\\end{{table}}
"""
    return latex_table

outliers_quantile = 0.35
filtered_eth = get_filtered(eth, outliers_quantile)
print(generate_latex_table(eth, filtered_eth, 'eth'))

filtered_btc = get_filtered(btc, outliers_quantile)
print(generate_latex_table(btc, filtered_btc, 'btc'))


\begin{table}[ht]
\centering
\begin{tabular}{@{}lcccccc@{}}
\Variable & \multicolumn{3}{c}{Diff IV (Options Characteristics)} & \multicolumn{3}{c}{Diff IV (Option Greeks)} \\
\cmidrule(r){2-4} \cmidrule(l){5-7}
& & & & & & \\
\midrule
Intercept & -18.7331 & -22.5288 & 6.6133 & 2.9514 & -29.1363 & -1.3671 \\
& (0.000) & (0.000) & (0.023) & (0.000) & (0.000) & (0.390) \\
\midrule
Call & -20.1958 & -19.7297 & -10.5328 &  &  &  \\
& (0.000) & (0.000) & (0.000) &  &  &  \\
Maturity & 0.0059 & -0.0163 & -0.2250 &  &  &  \\
& (0.517) & (0.125) & (0.000) &  &  &  \\
Mness & 41.3238 & 39.3198 & 18.4684 &  &  &  \\
& (0.000) & (0.000) & (0.000) &  &  &  \\
\midrule
Delta &  &  &  & 0.9989 & -15.6273 & -39.5663 \\
&  &  &  & (0.432) & (0.000) & (0.000) \\
Vega &  &  &  & 0.7540 & 6.4867 & 5.4907 \\
&  &  &  & (0.000) & (0.000) & (0.000) \\
\midrule
R-squared & 0.106 & 0.250 & 0.314 & 0.001 & 0.213 & 0.341 \\
Adj. R-squared & 0.106 & 0.246 & 0.309 & 0.001 & 0.209 & 0.337 \\
Observations & 21305 &